# SETUP

In [ ]:
import pickle
import sys
from glob import glob
from pathlib import Path

import numpy as np
import pandas as pd
import torch
import torchvision.transforms as T
from IPython.display import HTML, display
from matplotlib import pyplot as plt
from PIL import Image
from sklearn.metrics import (
    confusion_matrix,  # for evaluating the model of a single pixel basis
)
from torch.utils.data import DataLoader
from torchvision.utils import make_grid, save_image
from tqdm import tqdm

from src.max import *

%load_ext autoreload
%autoreload 2

pd.set_option("display.max_columns", None)

assert sys.version_info >= (
    3,
    10,
), "This notebook requires at least Python 3.10"

# DATA LOADING

In [ ]:
DATA_DIR = Path("data/CamVid/")

class_labels, X_train_paths, y_train_paths, X_test_paths, y_test_paths, X_val_paths, y_val_paths = locate_data(DATA_DIR, show=True)

class_labels['RGB'] = class_labels.apply(lambda x: (x['r'], x['g'], x['b']), axis=1)
class_labels = class_labels.drop(['r', 'g', 'b'], axis=1)

class_labels.T

In [ ]:
# Load data into memory
X_train = load_data(X_train_paths)
y_train = load_data(y_train_paths)

assert (
    type(X_train[0][0][0][0]) == np.uint8
), "image is not stored as unit8 dtype"  # PyTorch likes uint8

In [ ]:
# Show a sample image
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
img_idx = 0
ax1.set_title("Original Image")
ax1.imshow(X_train[img_idx])

ax2.set_title("Labeled Image")
ax2.imshow(y_train[img_idx])
plt.show()

In [ ]:
# Prep data for pytorch
training_batch_size = 20

train_loader = DataLoader(dataset=X_train, batch_size=training_batch_size)

# EDA

In [ ]:
height, width, color_channels = y_train[0].shape
print(f"there are {height * width} pixels in current image")

In [ ]:
# classes in current image
img_classes = np.unique(y_train[0].reshape(-1, y_train[0].shape[2]), axis=0)
img_classes

In [ ]:
# TODO iterate over all images

counts = {
    "Animal": 0,
    "Archway": 0,
    "Bicyclist": 0,
    "Bridge": 0,
    "Building": 0,
    "Car": 0,
    "CartLuggagePram": 0,
    "Child": 0,
    "Column_Pole": 0,
    "Fence": 0,
    "LaneMkgsDriv": 0,
    "LaneMkgsNonDriv": 0,
    "Misc_Text": 0,
    "MotorcycleScooter": 0,
    "OtherMoving": 0,
    "ParkingBlock": 0,
    "Pedestrian": 0,
    "Road": 0,
    "RoadShoulder": 0,
    "Sidewalk": 0,
    "SignSymbol": 0,
    "Sky": 0,
    "SUVPickupTruck": 0,
    "TrafficCone": 0,
    "TrafficLight": 0,
    "Train": 0,
    "Tree": 0,
    "Truck_Bus": 0,
    "Tunnel": 0,
    "VegetationMisc": 0,
    "Void": 0,
    "Wall": 0,
}

# count pixels for each class
for image in tqdm(y_train):
    for c, v in class_labels.to_dict()['RGB'].items():
        mask = np.all(image == v, axis=2)  # creates a boolean mask of current class
        class_pixel_count = np.sum(mask)
        counts.update({c: counts[c] + class_pixel_count})

# save counts
with open('dev_output/y_train_class_count.pkl', 'wb') as f:
    pickle.dump(counts, f)
    
counts

# MODEL DEV

Intersection over Union (IoU) -> commonly used evaluation metric for supervised semantic image segmentation tasks.

In [ ]:
SMOOTH = 1e-6 # To avoid division by zero


def iou(model_output: torch.Tensor, label: torch.Tensor) -> float:
    """Calculate the IoU for a single model output and label pair."""
    intersection = (
        (model_output & label).float().sum((1, 2))
    )  # Will be zero if Truth=0 or Prediction=0
    union = (model_output | label).float().sum((1, 2))  # Will be zero if both are 0
    
    iou = (intersection + SMOOTH) / (union + SMOOTH)  # smooth division to avoid 0/0
    
    thresholded = torch.clamp(20 * (iou - 0.7), 0, 10).ceil() / 10  # This is equal to comparing with thresolds
    
    return thresholded  # Or thresholded.mean() if you are interested in average across the batch


def iou_pytorch(outputs: torch.Tensor, labels: torch.Tensor):
    # You can comment out this line if you are passing tensors of equal shape
    # But if you are passing output from UNet or something it will most probably
    # be with the BATCH x 1 x H x W shape
    outputs = outputs.squeeze(1)  # BATCH x 1 x H x W => BATCH x H x W

    intersection = (outputs & labels).float().sum((1, 2))  # Will be zero if Truth=0 or Prediction=0
    union = (outputs | labels).float().sum((1, 2))         # Will be zzero if both are 0

    iou = (intersection + SMOOTH) / (union + SMOOTH)  # We smooth our devision to avoid 0/0

    thresholded = torch.clamp(20 * (iou - 0.7), 0, 10).ceil() / 10  # This is equal to comparing with thresolds

    return thresholded  # Or thresholded.mean() if you are interested in average across the batch

Training process:
1. calc loss
2. calc gradient
3. update weights
4. repeat